In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import math
import timeit
import utils
from tqdm import tqdm

import dowhy.datasets
import dowhy
from dowhy import CausalModel

from math import sqrt

from econml.drlearner import ForestDRLearner, LinearDRLearner
from econml.metalearners import SLearner, XLearner, TLearner
from econml.ortho_forest import CausalTree, ContinuousTreatmentOrthoForest, DiscreteTreatmentOrthoForest
from econml.dml import ForestDMLCateEstimator, LinearDMLCateEstimator, SparseLinearDMLCateEstimator
from econml.inference import BootstrapInference
from econml.sklearn_extensions.linear_model import WeightedLasso, WeightedLassoCV

### Import sklearn
from scipy.stats import sem
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import explained_variance_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from mlens.ensemble import SuperLearner

from zepid import load_sample_data, spline
from zepid.causal.doublyrobust import TMLE

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


In [2]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

RESULT_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/result"

In [4]:
expose_m = ' + '.join(cov)
outcome_m = ' + '.join(features)

In [69]:
expose_m

'x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8 + x9 + x10 + x11 + x12 + x13 + x14 + x15 + x16 + x17 + x18 + x19 + x20 + x21 + x22 + x23 + x24 + x25'

In [72]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

mae_train_seq = []
mae_test_seq = []

for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)
    model_t = LogisticRegression(penalty='l2', random_state=201)
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    
    tml = TMLE(data, exposure=treatment, outcome=outcome)
    tml.exposure_model(expose_m, bound=[0.01, 0.99], print_results=False, custom_model=model_t)
    tml.outcome_model(outcome_m, print_results=False, custom_model = model_y)
    tml.fit()
    
    true_effect = data['mu1'] - data['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    mae_train = utils.abs_ate(means, tml.average_treatment_effect)

    tml = TMLE(data_test, exposure=treatment, outcome=outcome)
    tml.exposure_model(expose_m, bound=[0.01, 0.99], print_results=False, custom_model=model_t)
    tml.outcome_model(outcome_m, print_results=False)
    tml.fit()
    
    true_effect = data_test['mu1'] - data_test['mu0']
    means, stds = np.mean(true_effect, axis=0), sem(true_effect, axis=0)
    mae_test = utils.abs_ate(means, tml.average_treatment_effect)
    
    mae_train_seq.append(mae_train)
    mae_test_seq.append(mae_test)
    


100%|██████████| 100/100 [08:31<00:00,  5.11s/it]


In [73]:
print(np.mean(mae_train_seq))

0.3285237069393208


In [74]:
print(np.mean(mae_test_seq))

1.1343649601008694
